# Short

In [1]:
from decimal import Decimal
from backintime.broker.base import OrderSide, MarketOrderOptions, TakeProfitOptions, StopLossOptions
from backintime.broker.futures.balance import FuturesBalanceManager

balance_manager = FuturesBalanceManager(Decimal('1501.24'), 
                                        per_contract_init_margin=Decimal('1500'),
                                        per_contract_fee=Decimal('0.62'))

# Enter Short position
balance_manager.hold_for_buy(MarketOrderOptions(order_side=OrderSide.SELL, amount=Decimal('1500')))
balance_manager.short_sell(Decimal('1500'), Decimal(1100))

# Hold position for TP/SL
balance_manager.hold_position(OrderSide.BUY, TakeProfitOptions(amount=Decimal('1500'), trigger_price=1000))
balance_manager.hold_position(OrderSide.BUY, StopLossOptions(amount=Decimal('1500'), trigger_price=1150))

# Close Short position
balance_manager.close_short(amount=Decimal(1500), fill_price=Decimal(1000))
balance_manager.balance

BalanceInfo(usd_balance=1700.00, available_usd_balance=1700.00, contracts=0, available_contracts=0)

## Going Short with Limit SELL & TP/SL

In [3]:
from decimal import Decimal
from datetime import datetime, timedelta
from backintime.broker.base import OrderSide, MarketOrderOptions, LimitOrderOptions, TakeProfitOptions, StopLossOptions
from backintime.broker.futures.broker import FuturesBroker

broker = FuturesBroker(Decimal('1501.24'), 
                       per_contract_init_margin=Decimal('1500'),
                       per_contract_fee=Decimal('0.62'))

tp_options = TakeProfitOptions(percentage_amount=Decimal('100'), trigger_price=Decimal('900'))
sl_options = StopLossOptions(percentage_amount=Decimal('100'), trigger_price=Decimal('1100'))

options = LimitOrderOptions(order_side=OrderSide.SELL, 
                            order_price=Decimal('1000'), 
                            amount=Decimal('1500'),
                            take_profit=tp_options,
                            stop_loss=sl_options)

from backintime.candles import CandlesBuffer, InputCandle
from backintime.timeframes import Timeframes as tf

start_time = datetime.fromisoformat('2020-01-01 00:00+00:00')
sample_candles = [    # our sample data
    InputCandle(open_time=start_time,
                open=Decimal(1000),
                high=Decimal(1050),
                low=Decimal(950),
                close=Decimal(1050),
                volume=Decimal(100),
                close_time=start_time + timedelta(seconds=59)),
    
    InputCandle(open_time=start_time,
                open=Decimal(900),
                high=Decimal(950),
                low=Decimal(850),
                close=Decimal(850),
                volume=Decimal(100),
                close_time=start_time + timedelta(seconds=59)),
]

short = broker.submit_limit_short(options)
for candle in sample_candles:
    broker.update(candle)

broker.balance

BalanceInfo(usd_balance=1700.00, available_usd_balance=1700.00, contracts=0, available_contracts=0)

## Going Short with Market SELL & close Short with Market BUY
It is not necessary to close Short position with TP/SL. Any BUY order can be used as well.

In [4]:
from decimal import Decimal
from datetime import datetime, timedelta
from backintime.broker.base import OrderSide, MarketOrderOptions, LimitOrderOptions, TakeProfitOptions, StopLossOptions
from backintime.broker.futures.broker import FuturesBroker

broker = FuturesBroker(Decimal('1501.24'), 
                       per_contract_init_margin=Decimal('1500'),
                       per_contract_fee=Decimal('0.62'))

sell_options = MarketOrderOptions(order_side=OrderSide.SELL, amount=Decimal('1500'))
buy_options = MarketOrderOptions(order_side=OrderSide.BUY, amount=Decimal('1500'))

from backintime.candles import CandlesBuffer, InputCandle
from backintime.timeframes import Timeframes as tf

start_time = datetime.fromisoformat('2020-01-01 00:00+00:00')
sample_candles = [    # our sample data
    InputCandle(open_time=start_time,
                open=Decimal(1000),
                high=Decimal(1050),
                low=Decimal(950),
                close=Decimal(1050),
                volume=Decimal(100),
                close_time=start_time + timedelta(seconds=59)),
    
    InputCandle(open_time=start_time,
                open=Decimal(900),
                high=Decimal(950),
                low=Decimal(850),
                close=Decimal(850),
                volume=Decimal(100),
                close_time=start_time + timedelta(seconds=59)),
]
# Submit Market Short
short = broker.submit_market_short(sell_options)
print(f"Market Short {short.status}: {broker.balance}")

# Market Sell is executed and Short position is opened
broker.update(sample_candles[0])
print(f"Market Short {short.status}: {broker.balance}")

# Submit Market BUY
buy = broker.submit_market_order(buy_options)
print(f"BUY back {buy.status}: {broker.balance}")

# Market BUY is executed and Short position is closed
broker.update(sample_candles[1])
print(f"BUY back {buy.status}: {broker.balance}")

print("------------------------------------")
print(f"Final balance: {broker.balance}")

Market Short CREATED: BalanceInfo(usd_balance=1501.24, available_usd_balance=0.62, contracts=0, available_contracts=0)
Market Short EXECUTED: BalanceInfo(usd_balance=1500.62, available_usd_balance=1500.62, contracts=0, available_contracts=0)
BUY back CREATED: BalanceInfo(usd_balance=1500.62, available_usd_balance=0.00, contracts=0, available_contracts=0)
BUY back EXECUTED: BalanceInfo(usd_balance=1700.00, available_usd_balance=1700.00, contracts=0, available_contracts=0)
------------------------------------
Final balance: BalanceInfo(usd_balance=1700.00, available_usd_balance=1700.00, contracts=0, available_contracts=0)
